In [1]:
import pandas as pd
import numpy as np

In [2]:
import re
import string

In [3]:
import datetime
import time

# Cleaning

In [4]:
nyangler_df = pd.read_csv('/home/desbrium/Metis/FishingSpots/Data/NyAngler/NYAnglerPostData06-22-21.csv')

nybass_df = pd.read_csv('/home/desbrium/Metis/FishingSpots/Data/NyBass/NYBassPostData06-21-21.csv')

In [5]:
nyangler_df.dropna(subset = ['Post_Content'], inplace = True)
nybass_df.dropna(subset = ['Post_Content'], inplace = True)

In [6]:
dt_pattern = r'(\w{3}\s\d+\,\s\d{4}.{9})'
find_date = re.compile(dt_pattern)

content_dt = [find_date.findall(content) for content in nybass_df['Post_Content']]

nybass_df['Post_Comment_Dates'] = content_dt

In [7]:
nybass_df['Post_Content'] = nybass_df['Post_Content'].apply(lambda x : re.sub(dt_pattern, '', x))

In [8]:
nyangler_df['Post_Content'] = nyangler_df['Post_Content'].str.replace('\n', '').str.replace('at ', '').str.lower()
nyangler_df['Post_Length'] = nyangler_df['Post_Content'].str.len()

nybass_df['Post_Content'] = nybass_df['Post_Content'].str.replace('\n', '').str.replace('at ', '').str.lower()
nybass_df['Post_Length'] = nybass_df['Post_Content'].str.len()

In [9]:
nyangler_df['Post_DateTime'] = nyangler_df['Post_DateTime'].str.replace('at ', '')
nyangler_df['Post_Date'] = nyangler_df['Post_DateTime'].apply(lambda x: x[:12].rstrip())
nyangler_df['Post_Time'] = nyangler_df['Post_DateTime'].apply(lambda x: x[12:].lstrip())
nyangler_df['Post_Year'] = nyangler_df['Post_Date'].apply(lambda x: x[-4:])
nyangler_df['Post_Month'] = nyangler_df['Post_Date'].apply(lambda x: x[:3])

nybass_df['Post_Date'] = nybass_df['Post_Date'].str.replace('at ', '')
nybass_df['Post_D'] = nybass_df['Post_Date'].apply(lambda x: x[:12].rstrip())
nybass_df['Post_Time'] = nybass_df['Post_Date'].apply(lambda x: x[12:].lstrip())
nybass_df['Post_Year'] = nybass_df['Post_D'].apply(lambda x: x[-4:])
nybass_df['Post_Month'] = nybass_df['Post_D'].apply(lambda x: x[:3])

In [10]:
nybass_mask = nybass_df['Thread_Name'].isin(['Places to Fish/Fishing Reports', 'Tackle and Fishing Tactics', 
                               'Multi-Species', 'Saltwater'])

nyangler_mask = nyangler_df['Thread_Name'].isin(['Fishing Report', 'New York State DEC Updates',
       'Inshore Tackle and Techniques with Lep', 'General Discussion',
       'Long Island Sound Fishing', 'Sweetwater - Lakes, Reservoirs and Ponds'])

In [11]:
# nyangler_df[['Post_Views', 'Post_Replies', 'Thread_Replies', 'Thread_Views']] = 'No Info'

# nyanglerbass_masked_df = pd.concat([nyangler_df[nyangler_mask].loc[:, ['Thread_Name', 'Post_Title', 'Post_Year', 'Post_Month', 'Post_Length', 'Post_Content', 'Post_Views', 'Post_Replies', 'Thread_Replies', 'Thread_Views']], 
#                                     nybass_df[nybass_mask].loc[:, ['Thread_Name', 'Post_Title', 'Post_Year', 'Post_Month', 'Post_Length', 'Post_Content', 'Post_Views', 'Post_Replies', 'Thread_Replies', 'Thread_Views']]], axis = 0)

In [12]:
nyanglerbass_masked_df = pd.concat([nyangler_df[nyangler_mask].loc[:, ['Thread_Name', 'Post_Title', 'Post_Year', 'Post_Month', 'Post_Length', 'Post_Content']], 
                                    nybass_df[nybass_mask].loc[:, ['Thread_Name', 'Post_Title', 'Post_Year', 'Post_Month', 'Post_Length', 'Post_Content', ]]], axis = 0)

In [13]:
nyanglerbass_masked_df['Post_Title'] = nyanglerbass_masked_df['Post_Title'].str.lower().str.strip()

In [14]:
nyanglerbass_masked_df = nyanglerbass_masked_df[~nyanglerbass_masked_df['Post_Title'].isin(['my joyce fishing montauk', 'my joyce montauk fishing'])]

In [15]:
def assign_season(month): 
    seasons = {'Spring' : ['Mar', 'Apr', 'May'], 'Summer': ['Jun', 'Jul', 'Aug'], 'Fall': ['Sep', 'Oct', 'Nov'], 'Winter': ['Dec', 'Jan', 'Feb']}
    
    for key, value in seasons.items():    
        if month in value:
            season = key
    return season

In [16]:
nyanglerbass_masked_df['Post_Season'] = nyanglerbass_masked_df['Post_Month'].apply(lambda month: assign_season(month))

In [17]:
# def lemmatize(document):
    
#     import spacy
#     nlp = spacy.load("en_core_web_sm")
    
#     new_doc = nlp(document)
#     new_doc_lemma = ' '.join([token.lemma_ for token in new_doc])
    
#     return new_doc_lemma

# lemmatized_doc = nyanglerbass_masked_df["Post_Content"].apply(lambda x: lemmatize(x))
# lemmatized_doc.to_csv('/home/desbrium/Metis/FishingSpots/Data/lemmatizeddocument.csv', index = False)

In [18]:
lemmatized_doc = pd.read_csv('/home/desbrium/Metis/FishingSpots/Data/lemmatizeddocument.csv')

nyanglerbass_masked_df = nyanglerbass_masked_df.reset_index()

nyanglerbass_masked_df['Lemma_Post_Content'] = lemmatized_doc
nyanglerbass_masked_df = nyanglerbass_masked_df.iloc[:, 1:]

In [19]:
nyanglerbass_masked_df["Lemma_Post_Content"] = nyanglerbass_masked_df["Lemma_Post_Content"].apply(lambda x: x.replace('flouro', 'fluoro'))
nyanglerbass_masked_df["Lemma_Post_Content"] = nyanglerbass_masked_df["Lemma_Post_Content"].apply(lambda x: x.replace('fluoro', 'fluorocarbon'))
nyanglerbass_masked_df["Lemma_Post_Content"] = nyanglerbass_masked_df["Lemma_Post_Content"].apply(lambda x: x.replace('fluorocarboncarbon', 'fluorocarbon'))

In [20]:
#nyanglerbass_masked_df.to_csv('/home/desbrium/Metis/FishingSpots/Data/nyanglerbass_maskedextra.csv', index = False)
nyanglerbass_masked_df.to_csv('/home/desbrium/Metis/FishingSpots/Data/nyanglerbass_masked.csv', index = False)

'/home/desbrium/Metis/FishingSpots/Data/nyanglerbass_masked.csv'